# CrewAI + Gaugid Integration

This notebook demonstrates how to integrate Gaugid SDK with CrewAI to build personalized multi-agent research teams that learn from user interactions.

In [ ]:
import os
import asyncio
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from gaugid import GaugidClient

# Set your API keys
os.environ["OPENAI_API_KEY"] = "your-key-here"  # or ANTHROPIC_API_KEY
os.environ["GAUGID_CONNECTION_TOKEN"] = "your-token-here"
os.environ["SERPER_API_KEY"] = "your-serper-key-here"  # Optional, for search

In [ ]:
async def load_user_context(client: GaugidClient) -> str:
    """Load user context from Gaugid profile using connection token mode."""
    # Connection token mode: No DID needed - profile resolved from token
    profile = await client.get_profile(
        scopes=["a2p:preferences", "a2p:professional", "a2p:context", "a2p:interests"]
    )
    
    # Extract memories from different memory types
    memories = profile.get("memories", {})
    episodic = memories.get("episodic", [])
    semantic = memories.get("semantic", [])
    procedural = memories.get("procedural", [])
    
    context_parts = []
    
    # Add episodic memories
    for memory in episodic:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Episodic: {category}] {content}")
    
    # Add semantic memories
    for memory in semantic:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Semantic: {category}] {content}")
    
    # Add procedural memories
    for memory in procedural:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Procedural: {category}] {content}")
    
    return "\n".join(context_parts) if context_parts else "No user context available."

# Initialize client
gaugid_client = GaugidClient(connection_token=os.environ["GAUGID_CONNECTION_TOKEN"])

# Load user context (connection token mode - no DID needed)
user_context = await load_user_context(gaugid_client)
print("User context loaded:")
print(user_context)

In [ ]:
# Create personalized research crew
researcher = Agent(
    role="Research Specialist",
    goal="Conduct thorough research on topics of interest to the user",
    backstory=f"""You are a research specialist with deep knowledge of the user's interests and expertise.

## USER PROFILE (from Gaugid)
{user_context}

You understand the user's professional background, interests, and learning goals.
Use this knowledge to tailor your research approach and find relevant information.""",
    verbose=True,
    allow_delegation=False,
    tools=[SerperDevTool()],
)

writer = Agent(
    role="Content Writer",
    goal="Create personalized, engaging content based on research findings",
    backstory=f"""You are a skilled content writer who creates personalized content.

## USER PROFILE (from Gaugid)
{user_context}

You adapt your writing style to match the user's preferences and expertise level.""",
    verbose=True,
    allow_delegation=False,
)

analyst = Agent(
    role="Data Analyst",
    goal="Analyze information and provide personalized insights",
    backstory=f"""You are a data analyst who provides personalized insights.

## USER PROFILE (from Gaugid)
{user_context}

You consider the user's professional background and interests when providing analysis.""",
    verbose=True,
    allow_delegation=False,
)

# Create tasks
research_task = Task(
    description="Research the topic provided by the user, considering their interests and expertise level",
    agent=researcher,
    expected_output="A comprehensive research report with key findings and sources",
)

writing_task = Task(
    description="Create personalized content based on the research findings",
    agent=writer,
    expected_output="Well-written, personalized content that matches the user's style",
)

analysis_task = Task(
    description="Analyze the research and content to provide personalized insights",
    agent=analyst,
    expected_output="Actionable insights and recommendations tailored to the user's needs",
)

# Create crew
crew = Crew(
    agents=[researcher, writer, analyst],
    tasks=[research_task, writing_task, analysis_task],
    process=Process.sequential,
    verbose=True,
)

print("✅ Personalized research crew ready with 3 specialized agents")

In [ ]:
# Execute research crew
topic = "Latest trends in AI agent frameworks"

print(f"Research topic: {topic}")
print("\n🔍 Crew is working...\n")

result = crew.kickoff(inputs={"topic": topic})

print("\n" + "="*60)
print("📊 CREW RESULTS")
print("="*60)
print(result)
print("="*60)

# Save the research session as a memory
memory_result = await gaugid_client.propose_memory(
    content=f"User requested research on: {topic}",
    category="a2p:interests.research",
    memory_type="episodic",
    confidence=0.8,
    context="CrewAI research session"
)
print(f"\nMemory saved: {memory_result.get('proposal_id')}")